In [104]:
# 股票代码格式统一

stock_concepts = pd.read_csv('./static/stock_concepts.csv',encoding="utf-8").fillna('')
stock_industry = pd.read_csv('./static/stock_industry.csv',encoding='gbk').fillna('')
stock_concepts['代码'] = stock_concepts['股票代码'].str.replace('.SH','').str.replace('.SZ','').str.replace('.BJ','')
stock_industry['代码'] = stock_industry['股票代码'].str.replace('.SH','').str.replace('.SZ','').str.replace('.BJ','')

#  全部股票

In [71]:
import wencai as wc
from wencai.core.session import Session

Session.headers.update({'Host':'www.iwencai.com'})
wc.set_variable(cn_col=True)

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)

index_selected_list = ['上证50','上证150','上证180','上证380','沪深300','科创50','创业板50',\
                       '中证500','中证1000','创业300','深证100','深证200','深证700']
concepts_list = pd.read_csv('./static/concepts_checklist.csv',encoding="gbk")
industry_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['行业','产品','业务','行业+']) & concepts_list['active']==1 ]['概念名称'].values.tolist()
events_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['事件','改革','政策']) & concepts_list['active']==1]['概念名称'].values.tolist()
master_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['知名企业','国企体系']) & concepts_list['active']==1]['概念名称'].values.tolist()

wc_result = pd.DataFrame()
for i in range(1,51):
    print('page: '+ str(i))
    sr = wc.search('概念 主营 所属指数类',page =i).fillna('')
    sr['沪深指数'] = ''
    sr['行业概念'] = ''
    sr['事件概念'] = ''
    sr['体系概念'] = ''

    for row, l in sr.iterrows():
        sr['主营产品名称'][row] = ','.join(l['主营产品名称'].split('||'))
        sr['沪深指数'][row] = ','.join(set(l['所属指数类'].split(';')).intersection(index_selected_list))
        sr['行业概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(industry_concepts_list)).replace('金属','')
        sr['事件概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(events_concepts_list))
        sr['体系概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(master_concepts_list))

    sr['所属概念'] =   sr['所属概念'].str.replace('概念股','').str.replace('概念','')
    sr['流动市值'] = (pd.to_numeric(sr['a股市值(不含限售股)'])/100000000).round(1).fillna('')    
    
    wc_result = wc_result.append(sr)
    
wc_result[['股票代码','股票简称','流动市值','最新价','最新涨跌幅','所属同花顺行业','行业概念','事件概念','体系概念','沪深指数','所属概念','主营产品名称']]\
            .to_csv('wc_info.csv',index = False, encoding = 'utf-8')

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27
page: 28
page: 29
page: 30
page: 31
page: 32
page: 33
page: 34
page: 35
page: 36
page: 37
page: 38
page: 39
page: 40
page: 41
page: 42
page: 43
page: 44
page: 45
page: 46
page: 47
page: 48
page: 49
page: 50


# 热点概念

In [ ]:
concept_list = []

for row, l in wc_result[wc_result['最新涨跌幅'].astype(float).between(9.9, 40, inclusive=False)].iterrows():
    concept_list = concept_list + l['行业概念'].split(',') + l['事件概念'].split(',') 
    concept_list =  [x for x in concept_list if x]   # remove ''
hot_concepts = pd.DataFrame(concept_list).apply(pd.value_counts)
hot_concepts = set(hot_concepts[hot_concepts[0] > 2].index)
hot_concepts